In [35]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [36]:
data1 = pd.read_csv("C:/Users/samba/OneDrive/Desktop/TAMU CORPUS/Driscoll/2021/digital twin/2021_avg_cc_0.csv")
data1

,day,average_cc_21_0
0,0cc21,0.002520
1,13cc21,0.473314
2,25cc21,14.547168
3,35cc21,21.776037
4,41cc21,29.348252
5,49cc21,46.459822
6,55cc21,62.859898
7,62cc21,76.352083
8,69cc21,89.949687
9,77cc21,97.915300


In [37]:
data2 = data1['average_cc_21_0']

In [44]:
dataset_train = data2[0:9]
dataset_test = data2[6:9]

In [45]:
training_set = dataset_train.values
training_set

array([2.52013300e-03, 4.73313898e-01, 1.45471678e+01, 2.17760368e+01,
       2.93482519e+01, 4.64598223e+01, 6.28598980e+01, 7.63520827e+01,
       8.99496872e+01])

In [46]:
testing_set = dataset_test.values
testing_set

array([62.85989802, 76.35208271, 89.94968717])

In [47]:
#vanilla lstm
# univariate lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# split into samples
X,y = list(), list()
for i in range(len(training_set)):
    #find the end of this pattern
    end_ix = i + n_steps_in
    out_end_ix = end_ix + n_steps_out
    #check if we are beyond the sequence
    if out_end_ix > len(training_set):
        break
    #gather input and output parts of the pattern
    seq_x, seq_y = training_set[i:end_ix], training_set[end_ix:out_end_ix]
    X.append(seq_x)
    y.append(seq_y)
X = np.array(X)
y = np.array(y)
print (X)
print (y)

[[2.52013300e-03 4.73313898e-01 1.45471678e+01]
 [4.73313898e-01 1.45471678e+01 2.17760368e+01]
 [1.45471678e+01 2.17760368e+01 2.93482519e+01]
 [2.17760368e+01 2.93482519e+01 4.64598223e+01]
 [2.93482519e+01 4.64598223e+01 6.28598980e+01]]
[[21.77603684 29.3482519 ]
 [29.3482519  46.45982227]
 [46.45982227 62.85989802]
 [62.85989802 76.35208271]
 [76.35208271 89.94968717]]


In [52]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
y = y.reshape((y.shape[0], y.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)

In [53]:
# demonstrate prediction
x_input = testing_set
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[[118.8398 ]
  [134.00621]]]
